In [4]:
import googlemaps
from datetime import datetime
import time  # Import the time module to add a delay between API requests
import dotenv
import os
import pickle
import math
import re
import demoji

dotenv.load_dotenv(dotenv.find_dotenv())
gmaps = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

# Geocoding an address
geocode_result = gmaps.geocode('國立陽明交通大學光復校區')

In [5]:
def get_result(keyword: str):
    results = []
    loc = geocode_result[0]['geometry']['location']
    # query_result = gmaps.places_nearby(keyword="餐廳",location=loc, rank_by='distance', language='zh-TW')
    query_result = gmaps.places_nearby(keyword=keyword,location=loc, radius=1500, language='zh-TW')
    results.extend(query_result['results'])
    while query_result.get('next_page_token'):
        time.sleep(2)
        query_result = gmaps.places_nearby(page_token=query_result['next_page_token'])
        results.extend(query_result['results'])    
    
    return results

In [6]:
queries = ['餐廳', 
           '早餐店', 
           '早午餐', 
           '牛排', 
           '火鍋', 
           '宵夜', 
           '炸物', 
           '燒烤', 
           '麵食', 
           '飲料', 
           '甜點', 
           '冰品',
           '咖啡', 
           '異國料理', 
           '台式料理',
           '泰式料理',
           '美式料理',
           '素食',
           '速食',
           '豆花',
           '滷味',
           '日式料理',
           '壽司',
           '義式餐廳',
           '食物', 
           '簡餐', 
           '餐廳', 
           '外帶',
           '麻辣', 
           '內用', 
           '吃到飽',
           '北方麵食',
           '熱炒',
           '拉麵',
           'pizza',
           '炸雞',
           '外送',
           '港式料理',
           '鴨肉',
           '雞肉',
           '羊肉',
           '鵝肉',
           '湯包',
           '鐵板燒',
           '便當',
           '焗烤',
           '炒飯',
           '小吃',
           '串燒',
           '牛肉']
results = []
for query in queries:
    results.extend(get_result(query))

In [7]:
results
# use pickle to save the results
with open('raw_results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [8]:
# use pickle to load the results
with open('raw_results.pickle', 'rb') as f:
    results = pickle.load(f)

In [9]:
current_loc = geocode_result[0]['geometry']['location']

# measure distance by lat and lng
def measure(loc1, loc2):  # generally used geo measurement function
    R = 6378.137  # Radius of earth in KM
    lat1, lon1 = loc1['lat'], loc1['lng']
    lat2, lon2 = loc2['lat'], loc2['lng']
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(lat1 * math.pi / 180) * math.cos(
        lat2 * math.pi / 180) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d # in km

# replace(if possible) and remove emoji
def replace_emoji(text):    
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, f' :{dem[item]}: ')
        
    text = re.sub(r'\\[Uu]000[a-zA-Z0-9]{5}', '', text) #.encode('unicode_escape').decode())
    # text = bytes(text, 'utf-8').decode('unicode_escape')
    return text

In [10]:
# delete duplicate
remain_keys = ['name', 'rating'] #, 'vicinity', 'place_id', 'geometry']

results = list({v['place_id']:v for v in results}.values())
for result in results:
    # place_detail = gmaps.place(result['place_id'], fields=['reviews', 'serves_breakfast', 'serves_brunch', 'serves_dinner', 'serves_lunch'], language='zh-TW')
    place_detail = gmaps.place(result['place_id'], fields=['reviews'], language='zh-TW')
    dest_loc = result['geometry']['location']
    
    for key in list(result.keys()):
        if key not in remain_keys: del result[key]
        
    result['distance'] = measure(current_loc, dest_loc)
    reviews = [review['text'] for review in place_detail['result']['reviews']] if place_detail['result'].get('reviews', False) else []
    reviews = [replace_emoji(review) for review in reviews] 
    result['reviews'] = reviews
    
    # result['serves_breakfast'] = place_detail['result'].get('serves_breakfast', False)
    # result['serves_brunch'] = place_detail['result'].get('serves_brunch', False)
    # result['serves_dinner'] = place_detail['result'].get('serves_dinner', False)
    # result['serves_lunch'] = place_detail['result'].get('serves_lunch', False)
    
    print(result['name'])

肥仔龍無煙鐵板燒
經濟部專業人員研究中心餐廳
野叔主廚私房料理
橄欖樹廚房
新天地美食館
儷舍坊
禧樂 丼飯、刺身、燒物
露米亞歐法餐廳
山威食堂
麥當勞-新竹光復餐廳
Westwood 現代美式餐廳
托斯卡尼尼義大利餐廳-竹科店
台積館品漾餐廳
LALA Kitchen 交大店
十六區和風料理
街巷Coffee&Restaurant
湘月食堂
泰式緬甸風味館
英美食坊
樂樂ENJOY PASTA
漢神鐵板燒／新竹鐵板燒／新竹美食／新竹餐廳／新竹必吃／光復路必吃／光復路美食
台灣光罩員工餐廳
RINGOAL車庫餐廳
水木餐廳
清華水漾餐廳
沐啡
和選旅-硯宴軒
禾園茶館
廚種 新竹學府店
瘋丼-新竹馬偕店-新竹外送美食|外送便當|日式丼飯|商務便當|推薦丼飯|必吃餐廳
明昌越式餐館-新竹美食-東區餐廳-河粉-越南料理-特色料理
山口刺身丼飯專賣店-清大店/新竹美食/清大夜市/生魚片/平價丼飯/日本料理
HOUSE KITCHEN 好事廚房｜LIFE KITCHEN 生活倉廚｜法式藜麥餐盒｜高級會議餐盒｜舒肥健康餐｜新竹商務便當｜輕食餐廳 ｜Takeout Restaurant｜Health Food Restaurant
謐‧義 La soffritto
璽子牛肉麵
原味燉品屋-建中店
國立陽明交通大學第一餐廳
哩賀巴西小館
車庫町日式咖哩，丼飯
饗食堂 麵飯飲料
新竹哈奇客韓式炸雞
好咖啡手工窯烤披薩
國立陽明交通大學第二餐廳
十上食堂
阿彤北和牛 海鮮 燒肉 定食 新竹總店
安妞韓國烤肉食堂-新竹竹科店
佬法室早午餐 Old Brunch
馬六甲馬來西亞風味餐廳清大店
老農家越式小館
也果蔬食 竹科店
初乎義料 chū hū Pasta
La stella義式人文餐廳
茉莉小鎮
費茲餐廳
川牛木石亭 光復店
米樂芙早午晚餐
清大雙喜餛飩、老虎麵
YOUKO 串燒き
風味小廚私房料理
麥當勞-新竹交大餐廳
來來豆漿店
永和豆漿大王
Friend’s早餐店
美之晨早餐屋
瑞斯飯糰清大店
美天美早餐漢堡店
早餐屋
g'day早餐店
美而佳早餐店
No. 19活力廚房
家鄉味早餐
8+8 早餐坊（高翠路175號）
幸福屋早餐坊
品香早餐
無名早餐店
拉亞漢堡 新竹建新
安迪早餐屋(安迪早午餐)
無名早餐店 饅頭肉排蛋
早安漢堡 小籠湯包
早餐屋
蘇朋傳統早餐
歐嗨唷(柚

In [11]:
results
# save processed result
with open('proc_results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [12]:
print(len(results))

568


In [13]:
results

[{'name': '肥仔龍無煙鐵板燒',
  'rating': 3.2,
  'distance': 1.2064086848736124,
  'reviews': ['2023/11\n很難想到現在還有120元可以吃到的鐵板燒\n店位在清華夜市裡\n旁邊有停車場（只能悠遊卡付款）\n店裡有兩大區料理平台\n看起來舊舊的\n平日中午1點左右到\n大概只坐了1/3\n\n點了學生套餐-牛肉（120元）包含豆芽菜、高麗菜、鯛魚片、牛肉片（不是學生也可以點學生套餐，另外有豬肉、羊肉可以選）\n點完餐後店家會送上白飯、濃湯、飲料杯\n濃湯、飲料喝到飽\n餐點味道都算正常\n豆芽菜、高麗菜都很脆口\n鯛魚片略乾\n牛肉片是一般黑胡椒口味\n每個人的餐點都集中放在一個盤子裡\n師傅料理完就直接堆疊起來\n會有味道互相干擾的情形\n個人比較不喜歡\n\n現場看不管是加辣、不吃豆芽菜換高麗菜等，都能配合客製調整，服務真的不錯\n\n整體來說CP值超高\n靠近學校的學生美食\n有順路經過值得一試',
   '便宜\n吃的很飽\n只是有點油膩\n畢竟要服務的人太多了\n一個學生餐，可能把附近餐飲打趴\n值得來試試CP值超高的本土鐵板燒',
   '可以吃得飽的地方\n還有120的學生餐，就有肉跟魚片\n來吃過就知道，什麼叫做吃飽\n店家真是佛心來的\n\n與對面的大埔完全不一樣的風格\n值得一試……',
   '每每經過人潮洶湧，絡繹不絕的～But.就是很不錯鐵板燒餐廳，雖然環境有太多不乾淨，但是呢，但是呢\n他們餐點是真的很好吃 :face savoring food: ，價格算可以，真的很好吃這一家也是跟同學甲粗飽唷！\n價格甜，餐點甜，吃飽也甜。\n這家店家甚麼都ok唯獨環境就～～需要改善了。',
   '有兩隻很可愛的小熊維尼師傅在炒菜\n只能說很油很鹹然後又很不乾淨\n但坦白來說在鐵板燒界這個價格真的算便宜了～\n\n120元（羊肉+豆芽+高麗菜+飯）另外濃湯紅茶喝到飽！非常划算！但沒有到很好吃\n\n看到把放在地上的桶子拿到鐵板上來放\n然後口罩沒戴好⋯\n\n肥仔龍\n吃了可能會變不健康的小肥仔']},
 {'name': '經濟部專業人員研究中心餐廳',
  'rating': 0,
  'distance': 0.7045454962066899,
  'rev

In [14]:
# read processed result
with open('proc_results.pickle', 'rb') as f:
    results = pickle.load(f)

In [15]:
# convert to csv
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('results.csv', index=False, encoding='utf-8-sig')

In [16]:
names = [result['name'] for result in results]
    
for name in names:
    print(name)

肥仔龍無煙鐵板燒
經濟部專業人員研究中心餐廳
野叔主廚私房料理
橄欖樹廚房
新天地美食館
儷舍坊
禧樂 丼飯、刺身、燒物
露米亞歐法餐廳
山威食堂
麥當勞-新竹光復餐廳
Westwood 現代美式餐廳
托斯卡尼尼義大利餐廳-竹科店
台積館品漾餐廳
LALA Kitchen 交大店
十六區和風料理
街巷Coffee&Restaurant
湘月食堂
泰式緬甸風味館
英美食坊
樂樂ENJOY PASTA
漢神鐵板燒／新竹鐵板燒／新竹美食／新竹餐廳／新竹必吃／光復路必吃／光復路美食
台灣光罩員工餐廳
RINGOAL車庫餐廳
水木餐廳
清華水漾餐廳
沐啡
和選旅-硯宴軒
禾園茶館
廚種 新竹學府店
瘋丼-新竹馬偕店-新竹外送美食|外送便當|日式丼飯|商務便當|推薦丼飯|必吃餐廳
明昌越式餐館-新竹美食-東區餐廳-河粉-越南料理-特色料理
山口刺身丼飯專賣店-清大店/新竹美食/清大夜市/生魚片/平價丼飯/日本料理
HOUSE KITCHEN 好事廚房｜LIFE KITCHEN 生活倉廚｜法式藜麥餐盒｜高級會議餐盒｜舒肥健康餐｜新竹商務便當｜輕食餐廳 ｜Takeout Restaurant｜Health Food Restaurant
謐‧義 La soffritto
璽子牛肉麵
原味燉品屋-建中店
國立陽明交通大學第一餐廳
哩賀巴西小館
車庫町日式咖哩，丼飯
饗食堂 麵飯飲料
新竹哈奇客韓式炸雞
好咖啡手工窯烤披薩
國立陽明交通大學第二餐廳
十上食堂
阿彤北和牛 海鮮 燒肉 定食 新竹總店
安妞韓國烤肉食堂-新竹竹科店
佬法室早午餐 Old Brunch
馬六甲馬來西亞風味餐廳清大店
老農家越式小館
也果蔬食 竹科店
初乎義料 chū hū Pasta
La stella義式人文餐廳
茉莉小鎮
費茲餐廳
川牛木石亭 光復店
米樂芙早午晚餐
清大雙喜餛飩、老虎麵
YOUKO 串燒き
風味小廚私房料理
麥當勞-新竹交大餐廳
來來豆漿店
永和豆漿大王
Friend’s早餐店
美之晨早餐屋
瑞斯飯糰清大店
美天美早餐漢堡店
早餐屋
g'day早餐店
美而佳早餐店
No. 19活力廚房
家鄉味早餐
8+8 早餐坊（高翠路175號）
幸福屋早餐坊
品香早餐
無名早餐店
拉亞漢堡 新竹建新
安迪早餐屋(安迪早午餐)
無名早餐店 饅頭肉排蛋
早安漢堡 小籠湯包
早餐屋
蘇朋傳統早餐
歐嗨唷(柚